In [179]:
import pandas as pd

In [180]:
df = pd.DataFrame()

In [181]:
from logger import Logger

In [182]:
loggerDf = Logger('LOGS.csv', mode='df')

In [183]:
import difflib
def show_diff(text, n_text):
    """
    http://stackoverflow.com/a/788780
    Unify operations between two compared strings seqm is a difflib.
    SequenceMatcher instance whose a & b are strings
    """
    seqm = difflib.SequenceMatcher(None, text, n_text)
    output= []
    for opcode, a0, a1, b0, b1 in seqm.get_opcodes():
        if opcode == 'equal':
            output.append(seqm.a[a0:a1])
        elif opcode == 'insert':
            output.append("<font color=red>^" + seqm.b[b0:b1] + "</font>")
        elif opcode == 'delete':
            output.append("<font color=blue>^" + seqm.a[a0:a1] + "</font>")
        elif opcode == 'replace':
            # seqm.a[a0:a1] -> seqm.b[b0:b1]
            output.append("<font color=green>^" + seqm.b[b0:b1] + "</font>")
        else:
            raise RuntimeError("unexpected opcode")
    return ''.join(output)

In [189]:
loggerDf.logsFile.sort_values(by='best-kappa')

,date,data-type,net-architecture,loss-func,optim,early-stopping-patience,parameters-amount,n-epochs,batch-size,best-train-loss,best-valid-loss,best-kappa,cfg,time_estimated,lb-kappa-score,dataset,trainloop,time-estimated,scheduler
12,2019.08.06 17:06:42,new_comp,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,15.0,44599361.0,20.0,64.0,271.764497,89.030222,0.0029,from model import MainModel\nfrom torch import...,662.72,NaN,NaN,NaN,NaN,NaN
29,2019.08.07 08:44:40,new_comp_quadratic_kappa,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,8.0,2099201.0,50.0,512.0,2.464634,2.461174,0.0038,from model import MainModel\nfrom torch import...,NaN,NaN,from torch.utils.data.dataset import Dataset\n...,import numpy as np # linear algebra\nimport pa...,NaN,NaN
42,2019.08.08 19:30:39,new_comp_quadratic_kappa,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,8.0,5320317.0,17.0,32.0,1.131912,1.052329,0.0224,from model import MainModel\nfrom torch import...,324.11,NaN,from torch.utils.data.dataset import Dataset\n...,import numpy as np # linear algebra\nimport pa...,NaN,NaN
13,2019.08.06 17:17:45,new_comp,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,15.0,44599361.0,63.0,64.0,0.466971,0.530520,0.2794,from model import MainModel\nfrom torch import...,2089.43,NaN,NaN,NaN,NaN,NaN
49,2019.08.11 17:45:02,new_old_mixed_ben_preprocessing,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,7.0,28342833.0,25.0,16.0,0.577896,0.685317,0.3629,from model import MainModel\nfrom torch import...,NaN,NaN,from torch.utils.data.dataset import Dataset\n...,import numpy as np # linear algebra\nimport pa...,NaN,<torch.optim.lr_scheduler.ReduceLROnPlateau ob...
50,2019.08.11 17:56:08,new_old_mixed_ben_preprocessing,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,8.0,28342833.0,30.0,16.0,0.577896,0.685317,0.3629,from model import MainModel\nfrom torch import...,NaN,NaN,from torch.utils.data.dataset import Dataset\n...,import numpy as np # linear algebra\nimport pa...,NaN,<torch.optim.lr_scheduler.ReduceLROnPlateau ob...
22,2019.08.06 22:28:37,new_comp_quadratic_kappa,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,15.0,44599361.0,100.0,64.0,0.803977,1.546088,0.3728,from model import MainModel\nfrom torch import...,NaN,NaN,NaN,NaN,NaN,NaN
46,2019.08.10 12:28:42,new_old_mixed,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,5.0,28346929.0,25.0,16.0,0.856866,0.839957,0.4322,from model import MainModel\nfrom torch import...,NaN,NaN,from torch.utils.data.dataset import Dataset\n...,import numpy as np # linear algebra\nimport pa...,NaN,NaN
66,2019.08.13 08:23:37,new_old_mixed_ben_preprocessing,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,8.0,28342833.0,100.0,16.0,0.723556,0.659867,0.4645,from model import MainModel\nfrom torch import...,NaN,NaN,from torch.utils.data.dataset import Dataset\n...,import numpy as np # linear algebra\nimport pa...,NaN,<torch.optim.lr_scheduler.ReduceLROnPlateau ob...
14,2019.08.06 17:52:35,new_comp,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,15.0,44599361.0,61.0,64.0,0.059520,0.324779,0.5353,from model import MainModel\nfrom torch import...,2023.55,NaN,NaN,NaN,NaN,NaN


In [135]:
loggerDf.logsFile.loc[63]

date                                                     2019.08.12 21:28:28
data-type                                                                new
net-architecture           import torch\nimport torch.nn as nn\nimport to...
loss-func                                                          MSELoss()
optim                      Adam (\nParameter Group 0\n    amsgrad: False\...
early-stopping-patience                                                    8
parameters-amount                                                2.83428e+07
n-epochs                                                                  22
batch-size                                                                16
best-train-loss                                                      0.20566
best-valid-loss                                                      0.25797
best-kappa                                                            0.8582
cfg                        from model import MainModel\nfrom torch import...

In [19]:
loggerDf.logsFile.loc[3, 'data-type'] = 'finetuning_new'
loggerDf.logsFile.loc[3, 'n-epochs'] = 50
loggerDf.logsFile.loc[3, 'batch-size'] = 16
loggerDf.logsFile.loc[3, 'time-estimated'] = 8100

,date,data-type,net-architecture,loss-func,optim,scheduler,early-stopping-patience,parameters-amount,n-epochs,batch-size,best-train-loss,best-valid-loss,best-kappa,lb-kappa-score,cfg,dataset,trainloop,time_estimated,time-estimated
0,2019.08.11 01:58:43,new_old_mixed_old,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,<torch.optim.lr_scheduler.CosineAnnealingLR ob...,5.0,28342833.0,11.0,16.0,0.651234,0.631161,0.2751,NaN,from model import MainModel\nfrom torch import...,from torch.utils.data.dataset import Dataset\n...,import numpy as np # linear algebra\nimport pa...,13907.77,NaN
1,2019.08.11 12:36:00,finetuning_new,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,<torch.optim.lr_scheduler.CosineAnnealingLR ob...,10.0,28342833.0,50.0,16.0,0.291845,0.305394,0.8427,NaN,from model import MainModel\nfrom torch import...,from torch.utils.data.dataset import Dataset\n...,import numpy as np # linear algebra\nimport pa...,4296.42,8100.0


In [23]:
loggerDf.save()

In [22]:
datetime.strftime(datetime.now(), "%Y.%m.%d %H:%M:%S")

'2019.08.02 00:51:32'

In [12]:
from libs.lr_finder import LRFinder


In [11]:
import torch

In [12]:
model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x16d_wsl')

Downloading: "https://github.com/facebookresearch/WSL-Images/archive/master.zip" to /home/paniquex/.cache/torch/hub/master.zip
Downloading: "https://download.pytorch.org/models/ig_resnext101_32x16-c6f796b0.pth" to /home/paniquex/.cache/torch/checkpoints/ig_resnext101_32x16-c6f796b0.pth
100%|██████████| 777518664/777518664 [01:08<00:00, 11411797.13it/s]


In [13]:
torch.save(model, 'resnext101_32x16d_wsl')

In [37]:
train_new = pd.read_csv('./input/train_new.csv')
train_old = pd.read_csv('./input/train_old.csv')

In [38]:
train_old

,id_code,diagnosis
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1
5,15_right,2
6,16_left,4
7,16_right,4
8,17_left,0
9,17_right,1


In [36]:
train_old.to_csv('./input/train_old.csv', index=False)

,id_code,diagnosis
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1
5,15_right,2
6,16_left,4
7,16_right,4
8,17_left,0
9,17_right,1
